<a href="https://colab.research.google.com/github/MWestberg3/bias-checker/blob/mwestberg%2Fbuild-data-loaders-and-rnn/bias_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import pandas as pd
import kagglehub as kh
import os

path = kh.dataset_download("subhankarpanda56/news-bias")

print("Path to dataset: ", path)

In [ ]:
# Find the CSV file within the downloaded directory:
for filename in os.listdir(path):
  if filename.endswith(".csv"):
    csv_file_path = os.path.join(path, filename)
    break # Stop after finding the first CSV file


bias_raw_df = pd.read_csv(csv_file_path).iloc[:300]
print(bias_raw_df.head())

In [ ]:
len(bias_raw_df)

In [ ]:
bias_raw_df.groupby('bias_text').count().plot(kind='pie', y='ID', autopct='%1.0f%%')

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(bias_raw_df['content_original'], bias_raw_df['bias_text'], test_size=0.2, random_state=42)

In [ ]:
X_train[0]

# NLTK Pre-processing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

# Tokenization
X_token_train = [word_tokenize(line)[:100] for line in X_train]
X_token_test = [word_tokenize(line)[:100] for line in X_test]

In [ ]:
print(X_token_train[0])

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# Lemmatization
lemmatizer = WordNetLemmatizer()
lemma_X_train = [[lemmatizer.lemmatize(word) for word in line] for line in X_token_train]
lemma_X_test = [[lemmatizer.lemmatize(word) for word in line] for line in X_token_test]
print(lemma_X_train[0])

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

# Remove stopwords
clear_X_train = [[word for word in line if word not in stopwords.words('english')] for line in lemma_X_train]
clear_X_test = [[word for word in line if word not in stopwords.words('english')] for line in lemma_X_test]
print(clear_X_train[0])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initiate a tfidf vectorizer object
vectorizer = TfidfVectorizer(preprocessor=' '.join)
# Fitting the training dataset and transform it
X_train_tfidf = vectorizer.fit_transform(clear_X_train)
# Transform the test dataset
X_test_tfidf = vectorizer.transform(clear_X_test)

print(X_train_tfidf.shape)

In [ ]:
print(X_train_tfidf[0])

In [ ]:
import gensim

D = gensim.corpora.Dictionary(clear_X_train)
print(D)

In [ ]:
train_indices = [[D.token2id[t] for t in line] for line in clear_X_train]
test_indices = [[D.token2id[t] if t in D else len(D) for t in line] for line in clear_X_test]

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

train_tensor = pad_sequence([torch.tensor(line) for line in train_indices],
                            batch_first=True,
                            padding_value=len(D)+2)
test_tensor = pad_sequence([torch.tensor(line) for line in test_indices],
                           batch_first=True,
                           padding_value=len(D)+2)

X_train_seq = nn.functional.one_hot(train_tensor)
X_test_seq = nn.functional.one_hot(test_tensor)

print(X_train_seq.shape)
print(X_test_seq.shape)

# Torch RNN Models

In [20]:
# Simple tensor operations
import torch

# Putting data into a tensor
X_train_tensor = torch.tensor(X_train_tfidf.toarray())
X_test_tensor = torch.tensor(X_test_tfidf.toarray())

In [24]:
import torch.nn as nn

# Using super to use the net like a layer (inherit nn.Module)
class RNNNet(nn.Module):
  def __init__(self, input_size, hidden_size=128): # Default hidden_layer size is 128 nodes
    super(RNNNet, self).__init__()
    self.hidden_size = hidden_size
    # Define the RNN layer as LSTM
    self.rnn_layer = nn.LSTM(input_size,
                             hidden_size,
                             batch_first=True)
    # Define the output layer
    self.output_layer = nn.Linear(hidden_size, 1)
    self.sigmoid_layer = nn.Sigmoid()

  def forward(self, x):
    # Initiate hidden layer h_0
    hidden_init = torch.zeros(2, x.shape[0], self.hidden_size)

    # Run the RNN layer
    output, _ = self.rnn_layer(x)

    # Given input, we need to pass the input through the layers
    return self.sigmoid_layer(self.output_layer(output[:,-1,:]))

In [26]:
lr = 0.01 # Learning rate
epochs = 70 # Training epochs

input_size = X_train_tensor.shape[-1] # Extract the last dimension

model = RNNNet(input_size).to("cuda:0")

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
